# RecordLinkage Linking - 247 & Rivals

> Leveraging the RL library to determine approximate matching over a range of fields using various string methods methods.  This specifically focuses on 247 & Rivals first.

In [ ]:
import json
import pandas
import time
import os
import recordlinkage
import csv
import core_constants as cc
import functions as fx

#not currently using jellyfish
import jellyfish as jf

### I don't like having all of this here - should push to functions

In [ ]:
outputDir = '..//scrapedData//'
field_agg = "_"

## Load the source file dict
sourceFiles = json.loads(open('..//config//sourceFiles.json', "r").read())

## Load the id config
idConfig = json.loads(open('..//config//idConfig.json', "r").read())

## Load and Merge Source Files Then Create a List of Dicts for each Dataset
> This was originally set up for all of the keys in the sourcefiles.json config.  Since this file is currently only going to serve 247 & Rivals, I've hardcoded the keys to fetch

In [ ]:
dataset_keys = ['sports247', 'rivals']
dfs = []
for key in dataset_keys:
    vars()[key] = fx.mergeSourceFiles (key, outputDir, sourceFiles)    
    dfs.append(vars()[key])

## Create New IDs
> This isn't elegant and I'd love to basically preprocess these collections prior to this notebook

In [ ]:
fx.createNewID(idConfig['sports247'], sports247, field_agg)
fx.createNewID(idConfig['rivals'], rivals, field_agg)

## Create Dataframes
> I might move these to their own pickles so I don't constantly have to recreate these each time

In [ ]:
df_sports247 = pandas.DataFrame(sports247)
df_rivals = pandas.DataFrame(rivals)

## Rename the indexes to something readable and predictable
> Otherwise these become 'index' and 'level_0', etc when you start to merge these.

In [ ]:
df_sports247.index.name = '247_index'
df_rivals.index.name = 'rivals_index'

## Create Blockers
> I swear this isn't working.  And honestly I guess I'm ok with it not working since I'm doing string operations later on this script.  This usually takes 20+ minutes - so it is doing something...

In [ ]:
indexer = recordlinkage.Index()
indexer.block('school')
indexer.block('year')
candidate_links = indexer.index(df_sports247, df_rivals)

## Define Features

> These are all pretty straightforward.  The toughest one to assess is position - since the services don't always categorize players in the same way or have the same abbreviation for a single position.  Since this is often only a 2 or 3 letter string, I decided to do an exact match.

In [ ]:
c = recordlinkage.Compare()

c.exact('ID', 'ID', label='ID')
c.string('playerName', 'playerName', method='damerau_levenshtein', label='playerName')
c.string('school', 'school', label='college')
c.string('city', 'city', label='city')
c.exact('state', 'state', label='state')
c.string('highSchool', 'highSchool', label='highSchool')
c.exact('position', 'position', label='position')
c.exact('year', 'year', label='year')

<Compare>

## Generate Comparison Vector Set and Output to Pickle
> This will take 60+ minutes to complete.  The output will be an 8+ GB pickle file

In [ ]:
features = c.compute(candidate_links, df_sports247, df_rivals)
features.to_pickle("features.pkl")